In [ ]:
# Instalar OpenCV
!pip install opencv-python-headless

# Instalar DeepFace
!pip install deepface

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
from deepface import DeepFace
from google.colab.patches import cv2_imshow

# Función para capturar la foto desde la webcam
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Redimensiona la salida para ajustarse al elemento de video.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Espera a que se haga clic en Capturar.
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

In [ ]:
from IPython.display import Image

# Inicializar el clasificador de rostros Haar
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

try:
    # Capturar la foto
    filename = take_photo()
    print('Saved to {}'.format(filename))

    # Mostrar la imagen capturada
    display(Image(filename))

    # Leer la imagen capturada usando OpenCV
    img = cv2.imread(filename)

    if img is not None:
        # Convertir a escala de grises para la detección de rostros
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Mostrar la imagen original
        print("Imagen original:")
        cv2_imshow(img)

        # Detectar rostros en la imagen
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            print("No se detectaron rostros en la imagen.")
        else:
            # Analizar cada rostro detectado
            for (x, y, w, h) in faces:
                rostro = img[y:y+h, x:x+w]

                try:
                    # Analizar el rostro para detectar emociones
                    result = DeepFace.analyze(rostro, actions=['emotion'], enforce_detection=False)
                    print(result)

                    # Obtener la emoción dominante
                    dominant_emotion = result[0]['dominant_emotion']
                    print(f"Emoción dominante: {dominant_emotion}")

                    # Dibujar un rectángulo alrededor del rostro y mostrar la emoción
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(img, dominant_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                except Exception as e:
                    print(f"No se pudo analizar el rostro: {e}")

            # Mostrar la imagen con las emociones detectadas
            cv2_imshow(img)

    else:
        print(f"No se pudo leer la imagen: {filename}")

except Exception as err:
    print(f"Error: {str(err)}")